In [2]:
from ultralytics import YOLO
import cv2
import time
import torch
import numpy as np

In [3]:
MODEL_PATH = "yolov8n.pt"

In [4]:
model = YOLO(MODEL_PATH)

In [5]:
if torch.cuda.is_available():
    model.to("cuda")
else:
    model.to("cpu")

In [6]:
camera = cv2.VideoCapture(0)

In [7]:
camera.release()

In [8]:
CLASS_NAMES = model.names

In [9]:
TARGET_CLASS_TRAFFIC_LIGHT = "traffic light"

In [18]:
camera = cv2.VideoCapture(0)
while True:
    ret, frame = camera.read()

    if not ret:
        print("Error while loading camera")
        break
    
    
    copyframe = frame.copy()
    results = model(frame, stream=False, verbose=False, conf=0.5)

    detected_traffic_lights = []

    for result in results:
        boxes = result.boxes
        for box in boxes:
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            class_name = CLASS_NAMES[cls]

            if class_name.lower() == TARGET_CLASS_TRAFFIC_LIGHT.lower():
                           
                roi = frame[y1:y2, x1:x2]

                hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

                avg_hue = np.mean(hsv[:,:,0])


                if avg_hue < 15 or avg_hue > 165:
                    light_color = "RED"
                    color = (0, 0, 255) # Red BGR
                elif 40 < avg_hue < 90:
                    light_color = "GREEN"
                    color = (0, 255, 0) # Green BGR
                else:
                    light_color = "UNKNOWN"
                    color = (0, 255, 255)

                detected_traffic_lights.append({
                    "box": (x1, y1, x2, y2),
                    "color": light_color,
                    "hue": avg_hue,
                    "confidence": conf,
                    "draw_color": color
                })


    for light in detected_traffic_lights:
             x1, y1, x2, y2 = light["box"]
             label = f"Light: {light['color']} ({light['hue']:.1f})"
             cv2.rectangle(copyframe, (x1, y1), (x2, y2), light["draw_color"], 2)
             cv2.putText(copyframe, label, (x1, y1 - 10),
                         cv2.FONT_HERSHEY_SIMPLEX, 0.5, light["draw_color"], 2)


    cv2.imshow("detected", copyframe)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()



            


